In [298]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
%matplotlib inline
import json
import datetime


from scipy.cluster.hierarchy import dendrogram, linkage

from sklearn.cluster import AgglomerativeClustering
#import scipy.cluster.hierarchy as sch

#import April12.ipynb
#%store -r icd_grouped 
import statsmodels.api as sm
from patsy import dmatrices 

# model selection
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV

import seaborn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer 

# Demographics

In [299]:
data2= pd.read_csv('ADMISSIONS.csv')
data2.head()
data2.shape # doesn't have empty rows
#data.head()
#data['SUBJECT_ID'].head(10)

(58976, 19)

In [300]:
data2.dtypes

ROW_ID                   int64
SUBJECT_ID               int64
HADM_ID                  int64
ADMITTIME               object
DISCHTIME               object
DEATHTIME               object
ADMISSION_TYPE          object
ADMISSION_LOCATION      object
DISCHARGE_LOCATION      object
INSURANCE               object
LANGUAGE                object
RELIGION                object
MARITAL_STATUS          object
ETHNICITY               object
EDREGTIME               object
EDOUTTIME               object
DIAGNOSIS               object
HOSPITAL_EXPIRE_FLAG     int64
HAS_CHARTEVENTS_DATA     int64
dtype: object

In [301]:
df = data2[['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ADMISSION_TYPE', 'INSURANCE', 'MARITAL_STATUS', 'ETHNICITY', 'DIAGNOSIS']]
df.head()




,ROW_ID,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,INSURANCE,MARITAL_STATUS,ETHNICITY,DIAGNOSIS
0,21,22,165315,EMERGENCY,Private,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE
1,22,23,152223,ELECTIVE,Medicare,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...
2,23,23,124321,EMERGENCY,Medicare,MARRIED,WHITE,BRAIN MASS
3,24,24,161859,EMERGENCY,Private,SINGLE,WHITE,INTERIOR MYOCARDIAL INFARCTION
4,25,25,129635,EMERGENCY,Private,MARRIED,WHITE,ACUTE CORONARY SYNDROME


# Label Encoding

In [302]:
le = LabelEncoder()
df['ADMISSION_TYPEc']= le.fit_transform(df['ADMISSION_TYPE']) 
df.head(10)
#df.loc[df['ADMISSION_TYPEc']=='4']

/Users/kathy908000/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ROW_ID,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,INSURANCE,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,ADMISSION_TYPEc
0,21,22,165315,EMERGENCY,Private,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,1
1,22,23,152223,ELECTIVE,Medicare,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0
2,23,23,124321,EMERGENCY,Medicare,MARRIED,WHITE,BRAIN MASS,1
3,24,24,161859,EMERGENCY,Private,SINGLE,WHITE,INTERIOR MYOCARDIAL INFARCTION,1
4,25,25,129635,EMERGENCY,Private,MARRIED,WHITE,ACUTE CORONARY SYNDROME,1
5,26,26,197661,EMERGENCY,Medicare,SINGLE,UNKNOWN/NOT SPECIFIED,V-TACH,1
6,27,27,134931,NEWBORN,Private,NaN,WHITE,NEWBORN,2
7,28,28,162569,ELECTIVE,Medicare,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0
8,29,30,104557,URGENT,Medicare,MARRIED,UNKNOWN/NOT SPECIFIED,UNSTABLE ANGINA\CATH,3
9,30,31,128652,EMERGENCY,Medicare,MARRIED,WHITE,STATUS EPILEPTICUS,1


onehotencoder = OneHotEncoder()
columnTransformer = ColumnTransformer([('encoder', 
                                        OneHotEncoder(), 
                                        [0])], 
                                      remainder='passthrough') 
df = np.array(columnTransformer.fit_transform(df), dtype = np.str) 


In [303]:
# creating instance of one-hot-encoder
enc = OneHotEncoder(handle_unknown='ignore')

# passing bridge-types-cat column (label encoded values of bridge_types)
enc_df = pd.DataFrame(enc.fit_transform(df[['ADMISSION_TYPEc']]).toarray())

enc_df.head(20)

,0,1,2,3
0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0
3,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0
5,0.0,1.0,0.0,0.0
6,0.0,0.0,1.0,0.0
7,1.0,0.0,0.0,0.0
8,0.0,0.0,0.0,1.0
9,0.0,1.0,0.0,0.0


In [304]:
# merge with main df bridge_df on key values
df = df.join(enc_df)
df.head(10)

,ROW_ID,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,INSURANCE,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,ADMISSION_TYPEc,0,1,2,3
0,21,22,165315,EMERGENCY,Private,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,1,0.0,1.0,0.0,0.0
1,22,23,152223,ELECTIVE,Medicare,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1.0,0.0,0.0,0.0
2,23,23,124321,EMERGENCY,Medicare,MARRIED,WHITE,BRAIN MASS,1,0.0,1.0,0.0,0.0
3,24,24,161859,EMERGENCY,Private,SINGLE,WHITE,INTERIOR MYOCARDIAL INFARCTION,1,0.0,1.0,0.0,0.0
4,25,25,129635,EMERGENCY,Private,MARRIED,WHITE,ACUTE CORONARY SYNDROME,1,0.0,1.0,0.0,0.0
5,26,26,197661,EMERGENCY,Medicare,SINGLE,UNKNOWN/NOT SPECIFIED,V-TACH,1,0.0,1.0,0.0,0.0
6,27,27,134931,NEWBORN,Private,NaN,WHITE,NEWBORN,2,0.0,0.0,1.0,0.0
7,28,28,162569,ELECTIVE,Medicare,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1.0,0.0,0.0,0.0
8,29,30,104557,URGENT,Medicare,MARRIED,UNKNOWN/NOT SPECIFIED,UNSTABLE ANGINA\CATH,3,0.0,0.0,0.0,1.0
9,30,31,128652,EMERGENCY,Medicare,MARRIED,WHITE,STATUS EPILEPTICUS,1,0.0,1.0,0.0,0.0


In [305]:
#rename onehotencoded columns
df = df.rename({0: 'ELECTIVE', 1: 'EMERGENCY', 2: 'NEWBORN', 3: 'URGENT'}, axis=1)
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,INSURANCE,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,ADMISSION_TYPEc,ELECTIVE,EMERGENCY,NEWBORN,URGENT
0,21,22,165315,EMERGENCY,Private,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,1,0.0,1.0,0.0,0.0
1,22,23,152223,ELECTIVE,Medicare,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1.0,0.0,0.0,0.0
2,23,23,124321,EMERGENCY,Medicare,MARRIED,WHITE,BRAIN MASS,1,0.0,1.0,0.0,0.0
3,24,24,161859,EMERGENCY,Private,SINGLE,WHITE,INTERIOR MYOCARDIAL INFARCTION,1,0.0,1.0,0.0,0.0
4,25,25,129635,EMERGENCY,Private,MARRIED,WHITE,ACUTE CORONARY SYNDROME,1,0.0,1.0,0.0,0.0


# Check weather groupby works

In [306]:
df.dtypes

ROW_ID               int64
SUBJECT_ID           int64
HADM_ID              int64
ADMISSION_TYPE      object
INSURANCE           object
MARITAL_STATUS      object
ETHNICITY           object
DIAGNOSIS           object
ADMISSION_TYPEc      int64
ELECTIVE           float64
EMERGENCY          float64
NEWBORN            float64
URGENT             float64
dtype: object

In [307]:
#round, remove the decimal points

df = df.astype({'ELECTIVE': int, 'EMERGENCY': int, 'NEWBORN': int, 'URGENT': int  })
df = df.astype({'ELECTIVE': str, 'EMERGENCY': str, 'NEWBORN': str, 'URGENT': str })

#df['ELECTIVE'].apply(np.ceil)

#df['ELECTIVE'] = df['ELECTIVE'].round(decimals=0)
#df['ELECTIVE'] = (df['ELECTIVE'] // 1e-5) * 1e-5
#df = df.astype ({'ELECTIVE': 'int' }) 
#df['ELECTIVE'] = df['ELECTIVE'].astype(int)
#df['ELECTIVE'] = (df['ELECTIVE'] * 1e2).astype(int) / 1e2

#df.style.set_precision(1)
#df.style.format({ 'ELECTIVE': '{:,.1f}'.format, 'EMERGENCY': '{:,.3f}'.format, })

#df.round()
#df = df.astype ({'ELECTIVE': 'int32' }) 
#df = df.astype ({'ELECTIVE': str }) 
#df['ELECTIVE'].head()
df.head(20)

,ROW_ID,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,INSURANCE,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,ADMISSION_TYPEc,ELECTIVE,EMERGENCY,NEWBORN,URGENT
0,21,22,165315,EMERGENCY,Private,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,1,0,1,0,0
1,22,23,152223,ELECTIVE,Medicare,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1,0,0,0
2,23,23,124321,EMERGENCY,Medicare,MARRIED,WHITE,BRAIN MASS,1,0,1,0,0
3,24,24,161859,EMERGENCY,Private,SINGLE,WHITE,INTERIOR MYOCARDIAL INFARCTION,1,0,1,0,0
4,25,25,129635,EMERGENCY,Private,MARRIED,WHITE,ACUTE CORONARY SYNDROME,1,0,1,0,0
5,26,26,197661,EMERGENCY,Medicare,SINGLE,UNKNOWN/NOT SPECIFIED,V-TACH,1,0,1,0,0
6,27,27,134931,NEWBORN,Private,NaN,WHITE,NEWBORN,2,0,0,1,0
7,28,28,162569,ELECTIVE,Medicare,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1,0,0,0
8,29,30,104557,URGENT,Medicare,MARRIED,UNKNOWN/NOT SPECIFIED,UNSTABLE ANGINA\CATH,3,0,0,0,1
9,30,31,128652,EMERGENCY,Medicare,MARRIED,WHITE,STATUS EPILEPTICUS,1,0,1,0,0


In [308]:
#check join 
ndf = df.groupby('SUBJECT_ID').agg({'ELECTIVE' : lambda x: ''.join(x)})
ndf.head(30)
ndf['ELECTIVE'].head(30)


SUBJECT_ID
2      0
3      0
4      0
5      0
6      1
7      0
8      0
9      0
10     0
11     0
12     1
13     0
16     0
17    10
18     0
19     0
20     1
21    00
22     0
23    10
24     0
25     0
26     0
27     0
28     1
30     0
31     0
32     1
33     0
34    00
Name: ELECTIVE, dtype: object

ndf.dtypes

#ndf = ndf.astype(str)
ndf = ndf.astype ({'ELECTIVE': str }) 


ndf.dtypes

df['ELECTIVE'] = ndf['ELECTIVE']
df.head(30)

#check regex
a  = 
#ndf.loc[ndf['ELECTIVE']str.contains('1'), ELECTIVE'] = 1

print(ndf)

In [309]:
elective = ndf['ELECTIVE']
elective.head(20)

SUBJECT_ID
2      0
3      0
4      0
5      0
6      1
7      0
8      0
9      0
10     0
11     0
12     1
13     0
16     0
17    10
18     0
19     0
20     1
21    00
22     0
23    10
Name: ELECTIVE, dtype: object

In [310]:
ndf.dtypes

ELECTIVE    object
dtype: object

In [311]:
#ndf = ndf.to_string()
#ndf = ndf.astype({'ELECTIVE': str })
ndf = ndf['ELECTIVE'].astype(str)


In [313]:
ndf.dtypes

dtype('O')

In [296]:
ndf

'SUBJECT_ID\n2                                                 0\n3                                                 0\n4                                                 0\n5                                                 0\n6                                                 1\n7                                                 0\n8                                                 0\n9                                                 0\n10                                                0\n11                                                0\n12                                                1\n13                                                0\n16                                                0\n17                                               10\n18                                                0\n19                                                0\n20                                                1\n21                                               00\n22                               

In [290]:

for i in range(46520):
    row  = elective.iloc[i]
    #if  row.contains('1'):
    if '1' in row: 
        ndf['ELECTIVE'][i] = '1'       
    else:
        ndf['ELECTIVE'][i] = '0'


KeyError: 'ELECTIVE'

In [249]:
ndf = ndf.sort_values(by=['SUBJECT_ID'])
ndf.head(30)

,ELECTIVE
SUBJECT_ID,
2,0
3,0
4,0
5,0
6,1
7,0
8,0
9,0
10,0


In [ ]:

arr1= []

for i in range(634709):
    if  group2.iloc[i]==1:
        arr1.append(1)
        
    else:
        arr1.append(0)

grouped['bg1'] = arr1

grouped.head()

In [4]:
data2.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [5]:
# multiple rows for each patient? yes
dup = data2[data2.duplicated(['SUBJECT_ID'])]
dup.tail(30)

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
58851,57780,96260,110058,2196-08-24 17:05:00,2196-10-03 14:25:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,LONG TERM CARE HOSPITAL,Private,ENGL,PROTESTANT QUAKER,SINGLE,BLACK/AFRICAN AMERICAN,NaN,NaN,STEVENS JOHNSON SYNDROME,0,1
58853,55962,90676,103021,2143-05-28 07:15:00,2143-06-05 13:30:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,REHAB/DISTINCT PART HOSP,Medicare,ENGL,JEWISH,MARRIED,WHITE,NaN,NaN,DRUG REFRACTORY ATRIAL FIBRILLATION\PULMONARY ...,0,1
58858,55967,90688,107472,2153-10-09 08:10:00,2153-10-12 17:31:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,2153-10-09 05:16:00,2153-10-09 09:18:00,PNEUMONIA,0,1
58859,55968,90688,127169,2154-05-16 07:15:00,2154-05-16 16:15:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,NaN,NaN,OSTEROARTHRITIS OF LEFT WRIST/SDA,0,0
58860,55969,90688,112686,2154-11-09 11:29:00,2154-11-13 17:30:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,LONG TERM CARE HOSPITAL,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,2154-11-09 05:33:00,2154-11-09 12:50:00,SEPSIS,0,1
58865,55974,90699,174425,2201-02-24 19:33:00,2201-03-02 16:30:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,ENGL,JEWISH,MARRIED,WHITE,2201-02-24 17:36:00,2201-02-24 20:22:00,HYDROCEPHALUS,0,1
58872,56884,93535,121562,2188-08-05 20:27:00,2188-08-07 16:48:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,Medicare,ENGL,JEWISH,SINGLE,WHITE,2188-08-05 17:56:00,2188-08-05 18:06:00,ASTHMA;COPD EXACERBATION,0,1
58878,56890,93560,140159,2156-06-11 07:30:00,2156-06-22 16:25:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,REHAB/DISTINCT PART HOSP,Private,ENGL,CATHOLIC,SINGLE,WHITE,NaN,NaN,"TRAUMATIC BRAIN HEMORRHAGE, LEFT /SDA",0,1
58886,57479,95372,171027,2123-04-22 17:14:00,2123-04-29 17:09:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,SPAN,OTHER,MARRIED,HISPANIC OR LATINO,2123-04-22 13:17:00,2123-04-22 19:05:00,ANEMIA/THROMBOCYTOPNEIA/FEVER,0,1
58887,57480,95372,128486,2123-05-16 12:20:00,2123-05-18 13:50:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicare,SPAN,OTHER,MARRIED,HISPANIC OR LATINO,2123-05-16 07:02:00,2123-05-16 14:54:00,WEAKNESS,0,1


In [6]:
#merged2.iloc[40000:40050]
data2.loc[data2['SUBJECT_ID']==23]

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1


In [7]:
data2.shape

(58976, 19)

In [8]:
# duplicate rows?, no
nodup = data2.drop_duplicates(subset=None, keep='first', inplace=False)
nodup.shape

(58976, 19)

In [9]:
data2 = data2.astype ({'ROW_ID': str }) 

In [10]:
#can'get group by with row ids, type error
data6 = data2.groupby('SUBJECT_ID').agg({'ROW_ID' : lambda x: ', '.join(x)})
data6.head(20)

,ROW_ID
SUBJECT_ID,
2,1
3,2
4,3
5,4
6,5
7,6
8,7
9,8
10,9


In [11]:
data3 = data2.drop_duplicates(subset='SUBJECT_ID', keep='first', inplace=False)
data3.shape


(46520, 19)

In [12]:
#Matrix Description
#ethinicty doesn't change, same patient duplicate droped, kept ethnicity in the top row 


#data3 = data2.drop_duplicates(subset='ETHNICITY' , keep='first', inplace=False)
#data3 = data2.groupby('SUBJECT_ID').agg({'ETHNICITY' : lambda x: ', '.join(x)})
#return first value for ethnicity not white twice 

data3 = data3[['SUBJECT_ID','ETHNICITY' ]]

data3.head(30)



,SUBJECT_ID,ETHNICITY
0,22,WHITE
1,23,WHITE
3,24,WHITE
4,25,WHITE
5,26,UNKNOWN/NOT SPECIFIED
6,27,WHITE
7,28,WHITE
8,30,UNKNOWN/NOT SPECIFIED
9,31,WHITE
10,32,WHITE


In [13]:
## group by each colummn and join? 

In [14]:
data4 = data2.groupby('SUBJECT_ID').agg({'INSURANCE' : lambda x: ', '.join(x)})
data4.head()

,INSURANCE
SUBJECT_ID,
2,Private
3,Medicare
4,Private
5,Private
6,Medicare


In [15]:
data5 = data2.groupby('SUBJECT_ID').agg({'ADMISSION_TYPE' : lambda x: ', '.join(x)})
data5.head()

,ADMISSION_TYPE
SUBJECT_ID,
2,NEWBORN
3,EMERGENCY
4,EMERGENCY
5,NEWBORN
6,ELECTIVE


In [16]:
demodata= pd.merge(data3, data4, how='inner', on='SUBJECT_ID')
demodata.head()

,SUBJECT_ID,ETHNICITY,INSURANCE
0,22,WHITE,Private
1,23,WHITE,"Medicare, Medicare"
2,24,WHITE,Private
3,25,WHITE,Private
4,26,UNKNOWN/NOT SPECIFIED,Medicare


In [17]:
demodata= pd.merge(demodata, data5, how='inner', on='SUBJECT_ID')
demodata.head()

,SUBJECT_ID,ETHNICITY,INSURANCE,ADMISSION_TYPE
0,22,WHITE,Private,EMERGENCY
1,23,WHITE,"Medicare, Medicare","ELECTIVE, EMERGENCY"
2,24,WHITE,Private,EMERGENCY
3,25,WHITE,Private,EMERGENCY
4,26,UNKNOWN/NOT SPECIFIED,Medicare,EMERGENCY


In [18]:
demodata= pd.merge(demodata, data6, how='inner', on='SUBJECT_ID')
demodata.head(30)

,SUBJECT_ID,ETHNICITY,INSURANCE,ADMISSION_TYPE,ROW_ID
0,22,WHITE,Private,EMERGENCY,21
1,23,WHITE,"Medicare, Medicare","ELECTIVE, EMERGENCY","22, 23"
2,24,WHITE,Private,EMERGENCY,24
3,25,WHITE,Private,EMERGENCY,25
4,26,UNKNOWN/NOT SPECIFIED,Medicare,EMERGENCY,26
5,27,WHITE,Private,NEWBORN,27
6,28,WHITE,Medicare,ELECTIVE,28
7,30,UNKNOWN/NOT SPECIFIED,Medicare,URGENT,29
8,31,WHITE,Medicare,EMERGENCY,30
9,32,WHITE,Medicaid,ELECTIVE,31


In [19]:
data2 = data2.astype({'DIAGNOSIS': str })     # ['group2'].astype(str)
#data = data.astype ({'ICD9_CODE': str })

In [20]:
data7 = data2.groupby('SUBJECT_ID').agg({'DIAGNOSIS' : lambda x: ', '.join(x)})
data7.head(40)

,DIAGNOSIS
SUBJECT_ID,
2,NEWBORN
3,HYPOTENSION
4,"FEVER,DEHYDRATION,FAILURE TO THRIVE"
5,NEWBORN
6,CHRONIC RENAL FAILURE/SDA
7,NEWBORN
8,NEWBORN
9,HEMORRHAGIC CVA
10,NEWBORN


In [21]:
demodata= pd.merge(demodata, data7, how='inner', on='SUBJECT_ID')
demodata.head(30)

,SUBJECT_ID,ETHNICITY,INSURANCE,ADMISSION_TYPE,ROW_ID,DIAGNOSIS
0,22,WHITE,Private,EMERGENCY,21,BENZODIAZEPINE OVERDOSE
1,23,WHITE,"Medicare, Medicare","ELECTIVE, EMERGENCY","22, 23",CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...
2,24,WHITE,Private,EMERGENCY,24,INTERIOR MYOCARDIAL INFARCTION
3,25,WHITE,Private,EMERGENCY,25,ACUTE CORONARY SYNDROME
4,26,UNKNOWN/NOT SPECIFIED,Medicare,EMERGENCY,26,V-TACH
5,27,WHITE,Private,NEWBORN,27,NEWBORN
6,28,WHITE,Medicare,ELECTIVE,28,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...
7,30,UNKNOWN/NOT SPECIFIED,Medicare,URGENT,29,UNSTABLE ANGINA\CATH
8,31,WHITE,Medicare,EMERGENCY,30,STATUS EPILEPTICUS
9,32,WHITE,Medicaid,ELECTIVE,31,TRACHEAL STENOSIS/SDA


In [22]:
demodata.loc[demodata['SUBJECT_ID']==36]

,SUBJECT_ID,ETHNICITY,INSURANCE,ADMISSION_TYPE,ROW_ID,DIAGNOSIS
13,36,WHITE,"Medicare, Medicare, Medicare","EMERGENCY, EMERGENCY, ELECTIVE","36, 37, 38",CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...


In [23]:
data7 = data2.groupby('SUBJECT_ID').agg({'ADMISSION_LOCATION' : lambda x: ', '.join(x)})
#data7.head()
demodata= pd.merge(demodata, data7, how='inner', on='SUBJECT_ID')
demodata.head()

,SUBJECT_ID,ETHNICITY,INSURANCE,ADMISSION_TYPE,ROW_ID,DIAGNOSIS,ADMISSION_LOCATION
0,22,WHITE,Private,EMERGENCY,21,BENZODIAZEPINE OVERDOSE,EMERGENCY ROOM ADMIT
1,23,WHITE,"Medicare, Medicare","ELECTIVE, EMERGENCY","22, 23",CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,"PHYS REFERRAL/NORMAL DELI, TRANSFER FROM HOSP/..."
2,24,WHITE,Private,EMERGENCY,24,INTERIOR MYOCARDIAL INFARCTION,TRANSFER FROM HOSP/EXTRAM
3,25,WHITE,Private,EMERGENCY,25,ACUTE CORONARY SYNDROME,EMERGENCY ROOM ADMIT
4,26,UNKNOWN/NOT SPECIFIED,Medicare,EMERGENCY,26,V-TACH,TRANSFER FROM HOSP/EXTRAM


In [24]:
data8 = data2.groupby('SUBJECT_ID').agg({'DISCHARGE_LOCATION' : lambda x: ', '.join(x)})
#data8.head()
demodata= pd.merge(demodata, data8, how='inner', on='SUBJECT_ID')
demodata.head()

,SUBJECT_ID,ETHNICITY,INSURANCE,ADMISSION_TYPE,ROW_ID,DIAGNOSIS,ADMISSION_LOCATION,DISCHARGE_LOCATION
0,22,WHITE,Private,EMERGENCY,21,BENZODIAZEPINE OVERDOSE,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H
1,23,WHITE,"Medicare, Medicare","ELECTIVE, EMERGENCY","22, 23",CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,"PHYS REFERRAL/NORMAL DELI, TRANSFER FROM HOSP/...","HOME HEALTH CARE, HOME HEALTH CARE"
2,24,WHITE,Private,EMERGENCY,24,INTERIOR MYOCARDIAL INFARCTION,TRANSFER FROM HOSP/EXTRAM,HOME
3,25,WHITE,Private,EMERGENCY,25,ACUTE CORONARY SYNDROME,EMERGENCY ROOM ADMIT,HOME
4,26,UNKNOWN/NOT SPECIFIED,Medicare,EMERGENCY,26,V-TACH,TRANSFER FROM HOSP/EXTRAM,HOME


In [25]:
data2.dtypes

ROW_ID                  object
SUBJECT_ID               int64
HADM_ID                  int64
ADMITTIME               object
DISCHTIME               object
DEATHTIME               object
ADMISSION_TYPE          object
ADMISSION_LOCATION      object
DISCHARGE_LOCATION      object
INSURANCE               object
LANGUAGE                object
RELIGION                object
MARITAL_STATUS          object
ETHNICITY               object
EDREGTIME               object
EDOUTTIME               object
DIAGNOSIS               object
HOSPITAL_EXPIRE_FLAG     int64
HAS_CHARTEVENTS_DATA     int64
dtype: object

In [26]:
data2 = data2.astype ({'MARITAL_STATUS': str }) 

In [27]:
data9 = data2.groupby('SUBJECT_ID').agg({'MARITAL_STATUS' : lambda x: ', '.join(x)})
#data8.head()
demodata= pd.merge(demodata, data9, how='inner', on='SUBJECT_ID')
demodata.head()

,SUBJECT_ID,ETHNICITY,INSURANCE,ADMISSION_TYPE,ROW_ID,DIAGNOSIS,ADMISSION_LOCATION,DISCHARGE_LOCATION,MARITAL_STATUS
0,22,WHITE,Private,EMERGENCY,21,BENZODIAZEPINE OVERDOSE,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,MARRIED
1,23,WHITE,"Medicare, Medicare","ELECTIVE, EMERGENCY","22, 23",CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,"PHYS REFERRAL/NORMAL DELI, TRANSFER FROM HOSP/...","HOME HEALTH CARE, HOME HEALTH CARE","MARRIED, MARRIED"
2,24,WHITE,Private,EMERGENCY,24,INTERIOR MYOCARDIAL INFARCTION,TRANSFER FROM HOSP/EXTRAM,HOME,SINGLE
3,25,WHITE,Private,EMERGENCY,25,ACUTE CORONARY SYNDROME,EMERGENCY ROOM ADMIT,HOME,MARRIED
4,26,UNKNOWN/NOT SPECIFIED,Medicare,EMERGENCY,26,V-TACH,TRANSFER FROM HOSP/EXTRAM,HOME,SINGLE


In [28]:
data2 = data2.astype ({'HADM_ID': str }) 

In [29]:
data10 = data2.groupby('SUBJECT_ID').agg({'HADM_ID' : lambda x: ', '.join(x)})
#data7.head()
demodata= pd.merge(demodata, data10, how='inner', on='SUBJECT_ID')
demodata.tail(50)

,SUBJECT_ID,ETHNICITY,INSURANCE,ADMISSION_TYPE,ROW_ID,DIAGNOSIS,ADMISSION_LOCATION,DISCHARGE_LOCATION,MARITAL_STATUS,HADM_ID
46470,98636,BLACK/AFRICAN AMERICAN,Medicaid,EMERGENCY,58537,ASTHMA;CHRONIC OBST PULM DISEASE,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,SINGLE,143528
46471,98639,WHITE,Medicare,ELECTIVE,58538,CONGESTIVE HEART FAILURE\Transcatheter Aortic ...,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,MARRIED,167385
46472,98640,WHITE,Private,EMERGENCY,58539,INTRACRANIAL HEMORRHAGE,CLINIC REFERRAL/PREMATURE,HOME,SINGLE,104073
46473,98642,WHITE,Medicare,EMERGENCY,58540,ACUTE CORONARY SYNDROME,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,SINGLE,162213
46474,98643,PATIENT DECLINED TO ANSWER,Private,ELECTIVE,58541,ATRIAL FIBILLATION\BILATERAL THORACOSCOPIC MIN...,PHYS REFERRAL/NORMAL DELI,HOME,MARRIED,177932
46475,98644,BLACK/AFRICAN AMERICAN,Medicare,EMERGENCY,58542,ACUTE RENAL FAILURE,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,MARRIED,109643
46476,98647,WHITE,Medicare,EMERGENCY,58543,LEFT FEMUR FRACTURE,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,SINGLE,167391
46477,98649,ASIAN - ASIAN INDIAN,Medicaid,EMERGENCY,58544,PNEUMONIA,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,WIDOWED,110990
46478,98656,WHITE,Medicare,EMERGENCY,58545,ABDOMINAL PAIN,CLINIC REFERRAL/PREMATURE,LONG TERM CARE HOSPITAL,WIDOWED,138450
46479,98664,WHITE,Private,EMERGENCY,58546,CHEST PAIN,EMERGENCY ROOM ADMIT,SNF,SEPARATED,150793


In [30]:
# match with ICD9 code using HADMID? 
demodata.shape

(46520, 10)

# Check if Regex to find row works

In [31]:
sample = demodata.loc[demodata['SUBJECT_ID']==34]
sample['ROW_ID']

11    33, 34
Name: ROW_ID, dtype: object

In [33]:
#doesn't work
#demodata.loc[demodata['ROW_ID']==[33,34]]


In [37]:
#change groupby to make 1 and 0 regex work, no comma, a string
sample2 = data2.groupby('SUBJECT_ID').agg({'ADMISSION_TYPE' : lambda x: ''.join(x)})
data5.head(30)

,ADMISSION_TYPE
SUBJECT_ID,
2,NEWBORN
3,EMERGENCY
4,EMERGENCY
5,NEWBORN
6,ELECTIVE
7,NEWBORN
8,NEWBORN
9,EMERGENCY
10,NEWBORN


In [33]:
#sample2.loc[sample2['ADMISSION_TYPE'].str.contains('NEW'), 'ADMISSION_TYPE'] = 1
#sample2.head(10)

#rows = sample2[sample2['ADMISSION_TYPE'].str.contains('NEW')]
#dataframe with selected rows 
#rows['ADMISSION_TYPE'].iloc['']

#df.b.str.contains('^f')

#groups to int

#only numeric columns datamap['group'] = datamap['group'].str.replace('V','10',regex=True) datamap['group'] = datamap['group'].str.replace('E','11',regex=True)

#datamap['group'] = datamap['group'].astype(np.int) datamap['group'] = datamap['group'].astype(np.float) datamap.head(30)

In [49]:
#with random dataframe
train = pd.DataFrame({'Cabin':[10,20,30,40,50], 'Pclass':[1,2,3,1,2]})
print (train)

   Cabin  Pclass
0     10       1
1     20       2
2     30       3
3     40       1
4     50       2


In [50]:
train.loc[train['Pclass'] == 1, 'Pclass'] = 2
print(train)

   Cabin  Pclass
0     10       2
1     20       2
2     30       3
3     40       2
4     50       2


# Age

In [161]:
agedata= pd.read_csv('PATIENTS.csv')
agedata.shape

(46520, 8)

In [162]:
#admission and patient.csv date of admission - doB

# LabelEncoding

In [163]:
le = LabelEncoder() 

In [164]:
demodata['ETHNICITYc']= le.fit_transform(demodata['ETHNICITY']) 

demodata.head()

,SUBJECT_ID,ETHNICITY,INSURANCE,ADMISSION_TYPE,ROW_ID,DIAGNOSIS,ADMISSION_LOCATION,DISCHARGE_LOCATION,MARITAL_STATUS,HADM_ID,ETHNICITYc
0,22,WHITE,Private,EMERGENCY,21,BENZODIAZEPINE OVERDOSE,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,MARRIED,165315,36
1,23,WHITE,"Medicare, Medicare","ELECTIVE, EMERGENCY","22, 23",CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,"PHYS REFERRAL/NORMAL DELI, TRANSFER FROM HOSP/...","HOME HEALTH CARE, HOME HEALTH CARE","MARRIED, MARRIED","152223, 124321",36
2,24,WHITE,Private,EMERGENCY,24,INTERIOR MYOCARDIAL INFARCTION,TRANSFER FROM HOSP/EXTRAM,HOME,SINGLE,161859,36
3,25,WHITE,Private,EMERGENCY,25,ACUTE CORONARY SYNDROME,EMERGENCY ROOM ADMIT,HOME,MARRIED,129635,36
4,26,UNKNOWN/NOT SPECIFIED,Medicare,EMERGENCY,26,V-TACH,TRANSFER FROM HOSP/EXTRAM,HOME,SINGLE,197661,35


In [167]:
demodata['INSURANCEc']= le.fit_transform(demodata['INSURANCE']) 
demodata.head(10)

,SUBJECT_ID,ETHNICITY,INSURANCE,ADMISSION_TYPE,ROW_ID,DIAGNOSIS,ADMISSION_LOCATION,DISCHARGE_LOCATION,MARITAL_STATUS,HADM_ID,ETHNICITYc,INSURANCEc
0,22,WHITE,Private,EMERGENCY,21,BENZODIAZEPINE OVERDOSE,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,MARRIED,165315,36,154
1,23,WHITE,"Medicare, Medicare","ELECTIVE, EMERGENCY","22, 23",CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,"PHYS REFERRAL/NORMAL DELI, TRANSFER FROM HOSP/...","HOME HEALTH CARE, HOME HEALTH CARE","MARRIED, MARRIED","152223, 124321",36,105
2,24,WHITE,Private,EMERGENCY,24,INTERIOR MYOCARDIAL INFARCTION,TRANSFER FROM HOSP/EXTRAM,HOME,SINGLE,161859,36,154
3,25,WHITE,Private,EMERGENCY,25,ACUTE CORONARY SYNDROME,EMERGENCY ROOM ADMIT,HOME,MARRIED,129635,36,154
4,26,UNKNOWN/NOT SPECIFIED,Medicare,EMERGENCY,26,V-TACH,TRANSFER FROM HOSP/EXTRAM,HOME,SINGLE,197661,35,98
5,27,WHITE,Private,NEWBORN,27,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,nan,134931,36,154
6,28,WHITE,Medicare,ELECTIVE,28,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,MARRIED,162569,36,98
7,30,UNKNOWN/NOT SPECIFIED,Medicare,URGENT,29,UNSTABLE ANGINA\CATH,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,MARRIED,104557,35,98
8,31,WHITE,Medicare,EMERGENCY,30,STATUS EPILEPTICUS,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,MARRIED,128652,36,98
9,32,WHITE,Medicaid,ELECTIVE,31,TRACHEAL STENOSIS/SDA,PHYS REFERRAL/NORMAL DELI,HOME,nan,175413,36,35


# Diagnosis

In [36]:
data= pd.read_csv('group19gender.csv')
data.shape # doesn't have empty rows
#data.head()
#data['SUBJECT_ID'].head(10)


/Users/kathy908000/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (1,4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(634709, 35)

In [37]:
data.loc[6:11].head(10)

,ROW_ID_x,ICD9_CODE,SHORT_TITLE_x,LONG_TITLE_x,group,group2,lv.1_code,lv.2_code,lv.1_group,lv.2_group,...,bg10,bg11,bg12,bg13,bg14,bg15,bg16,bg17,bg18,bg19
6,252.0,41,Shigella flexneri,Shigella flexneri,0,1.0,001-009,4.0,INTESTINAL INFECTIOUS DISEASES,Shigellosis,...,0,0,0,0,0,0,0,0,0,0
7,252.0,41,Shigella flexneri,Shigella flexneri,0,1.0,001-009,4.0,INTESTINAL INFECTIOUS DISEASES,Shigellosis,...,0,0,0,0,0,0,0,0,0,0
8,252.0,41,Shigella flexneri,Shigella flexneri,0,1.0,001-009,4.0,INTESTINAL INFECTIOUS DISEASES,Shigellosis,...,0,0,0,0,0,0,0,0,0,0
9,255.0,48,Shigella infection NEC,Other specified shigella infections,0,1.0,001-009,4.0,INTESTINAL INFECTIOUS DISEASES,Shigellosis,...,0,0,0,0,0,0,0,0,0,0
10,256.0,49,Shigellosis NOS,"Shigellosis, unspecified",0,1.0,001-009,4.0,INTESTINAL INFECTIOUS DISEASES,Shigellosis,...,0,0,0,0,0,0,0,0,0,0
11,258.0,51,Botulism food poisoning,Botulism food poisoning,0,1.0,001-009,5.0,INTESTINAL INFECTIOUS DISEASES,Other food poisoning (bacterial),...,0,0,0,0,0,0,0,0,0,0


In [38]:
data = data.drop_duplicates(subset=None, keep='first', inplace=False)
data.shape

(634649, 35)

# Each Row Patient

In [39]:
#data['group2'] = data['group2'].astype(str)
data = data.astype({'group2': str })     # ['group2'].astype(str)
data = data.astype ({'ICD9_CODE': str }) 

In [40]:
data.dtypes

ROW_ID_x         float64
ICD9_CODE         object
SHORT_TITLE_x     object
LONG_TITLE_x      object
group             object
group2            object
lv.1_code         object
lv.2_code        float64
lv.1_group        object
lv.2_group        object
SUBJECT_ID       float64
HADM_ID          float64
GENDER             int64
DOB               object
DOD              float64
DOD_HOSP          object
bg1                int64
bg2                int64
bg3                int64
bg4                int64
bg5                int64
bg6                int64
bg7                int64
bg8                int64
bg9                int64
bg10               int64
bg11               int64
bg12               int64
bg13               int64
bg14               int64
bg15               int64
bg16               int64
bg17               int64
bg18               int64
bg19               int64
dtype: object

In [41]:
#group by groupconcacat
#data = data.groupby('SUBJECT_ID').agg({'group2' : lambda x: ', '.join(x)})
#data = data.groupby('SUBJECT_ID').agg({'group2' : lambda x: ', '.join(x)}, {'ICD9_CODE' :lambda x: ', '.join(x) })

#data = data.groupby('SUBJECT_ID').agg({'group2' : lambda x: ', '.join(x), 'ICD9_CODE' :lambda x: ', '.join(x) })
#data.head()

In [42]:
pdata = data.groupby('SUBJECT_ID')['ICD9_CODE'].apply(lambda group_series: group_series.tolist()).reset_index()
pdata.head(10)


,SUBJECT_ID,ICD9_CODE
0,2.0,"[V053, V290, V3001]"
1,3.0,"[389, 2639, 41071, 4254, 4275, 4280, 5849, 682..."
2,4.0,"[4111, 42, 1363, 2763, 5715, 7994, 7907, E9317..."
3,5.0,"[V053, V290, V3000]"
4,6.0,"[2753, 2767, 2859, 40391, 9972, V1582]"
5,7.0,"[V053, V290, V3001]"
6,8.0,"[7706, 7746, V053, V290, V3001, V502]"
7,9.0,"[4019, 4280, 431, 5070, 5849]"
8,10.0,"[76525, 76515, 7742, V290, V3000]"
9,11.0,[1913]


In [43]:
#to group based on ICD9-code, one hot encoding each category first 2 digits

#pdata['2dgroup'] = list(map(lambda x: x[0:2], list(pdata['ICD9_CODE'])))
#pdata.head()

# find all diagnoses of single patient, make a list, make new dataframe

In [44]:
icd9 = pdata['ICD9_CODE']
icd9

0                                      [V053, V290, V3001]
1        [389, 2639, 41071, 4254, 4275, 4280, 5849, 682...
2        [4111, 42, 1363, 2763, 5715, 7994, 7907, E9317...
3                                      [V053, V290, V3000]
4                   [2753, 2767, 2859, 40391, 9972, V1582]
                               ...                        
46512    [389, 23871, 2449, 2724, 2859, 48241, 4870, 51...
46513    [389, 25000, 27651, 40291, 4280, 5538, 5570, 5...
46514    [2721, 2851, 3004, 4019, 53081, 56881, 5699, 5...
46515    [25000, 2724, 2851, 41401, 4241, 4280, 42833, ...
46516                     [25000, 4019, 4139, 75612, 7861]
Name: ICD9_CODE, Length: 46517, dtype: object

digit = []
row = []
a = []

for i in range(634650):
    for j in icd9.iloc[i]:
        digit = j[0:2]
        row.append(digit)
    a.append(row)

a
    
